In [9]:
import numpy as np
from mbptycho.code.simulation import SampleParams, SimulationParams
from typing import List, Tuple

In [3]:
def HKLToTheta(lattice_dist: np.array, wavelength: float, H: int, K: int, L: int):
    """Adapt input for Laue geometry at HXN from APS nanoprobe mounting convention.

    Output is in theta, tth, gam APS NP convention: tth in-plane, gam oop

    Parameters
    ----------
    H : int
    K : int
    L : int
    Returns
    -------
    theta : float
        Theta angle in radians
    two_theta : float
        Two-theta in radians
    gamma : float
        Gamma in radians
    """
    Htemp = K
    Ktemp = L
    Ltemp = H

    H = Htemp
    L = Ltemp
    K = Ktemp

    ang_y = np.arctan2(L, H)
    # what is ang_sym?
    ang_sym = np.arcsin((np.sqrt(H ** 2 + K ** 2 + L ** 2) / 2) / (lattice_dist[0] /  wavelength))
    ang2_sym = 2 * ang_sym

    det_phi = np.arcsin(np.cos(ang_y) / np.sin(np.pi / 2 - ang_sym))
    temp = (np.sin(ang2_sym) * np.cos(det_phi)) ** 2 + np.cos(ang2_sym) ** 2

    theta = np.arcsin(np.cos(np.pi / 2 - ang_sym) / np.sin(ang_y))
    two_theta = np.arcsin(np.sin(ang2_sym) * np.cos(det_phi) / temp ** 0.5)
    gamma = np.arctan(np.sin(ang2_sym) * np.sin(det_phi) / temp ** 0.5)

    return np.array([theta, two_theta, gamma])

In [10]:
def getRotations(theta: float, two_theta: float, gamma: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Get rotation matrices corresponding to the provided motor positions.

    The angles provided (and the rotations calculated) do not use the standard clockwise/counterclockwise
    convention. I adapted Stephan's code as is, but I need to check with him to make sure what I am doing here
    actually makes sense. Also need to document these rotations properly.

    Parameters
    ----------
    theta : float
        Theta motor (in radians) (which corresponds to a rotation about the y-axis).
    two_theta : float
        Two-theta motor (in radians)
    gamma : float
        Gamma motor (in radians)
    Returns
    -------
    rotate_theta : array(float)
        3x3 Matrix that gives the rotation for the theta angle (about the y-axis). See `Notes`.
    rotate_two_theta : array(float)
        3x3 rotation matrix for two_theta
    rotate_gamma : array(float)
        3x3 rotation matrix for gamma
    Notes
    -----
    Assumes that the coordinates are stored in (y,x,z) format rather than the typical (x,y,z) format.
    """

    # this is rotation about the y-axis
    rotate_theta = np.array([[1, 0, 0],
                             [0, np.cos(theta), np.sin(theta)],
                             [0, -np.sin(theta), np.cos(theta)]])

    rotate_two_theta = np.array([[1, 0, 0],
                                 [0, np.cos(-two_theta), np.sin(-two_theta)],
                                 [0, -np.sin(-two_theta), np.cos(-two_theta)]])

    rotate_gamma = np.array([[np.cos(-gamma), 0, -np.sin(-gamma)],
                             [0, 1, 0],
                             [np.sin(-gamma), 0, np.cos(-gamma)]])
    return rotate_theta, rotate_two_theta, rotate_gamma

In [6]:

sim_params = SimulationParams()
sample_params = SampleParams(sim_params.sample_pix_size, sim_params.wavelength)
hkl_list = np.array([[1, 0, 0], [1, 1, 0], [1, 2, 0], [2, 1, 0]])
motor_list = np.array([-HKLToTheta(sample_params.lattice, sim_params.wavelength, *HKL)  for HKL in hkl_list])

In [30]:
thetas_rad, two_thetas_rad, gammas_rad

(array([-0.17731696, -0.36053885, -1.07997542, -0.45668261]),
 array([-0.35463393, -0.36053885, -0.23685368, -0.74335671]),
 array([-2.16014524e-17, -3.60538849e-01, -7.83208806e-01, -3.60538849e-01]))

In [32]:
thetas_rad, two_thetas_rad, gammas_rad = motor_list.T
q_norm_all = np.zeros(hkl_list.shape[0])
for i in range(hkl_list.shape[0]):
    rotate_theta, rotate_two_theta, rotate_gamma = getRotations(thetas_rad[i], 
                                                                two_thetas_rad[i],
                                                                gammas_rad[i])
    print(i)
    print(rotate_theta)
    ki = np.array([0, 0, 1])[:, None]

    ki_rotated = rotate_gamma @ rotate_two_theta @ ki
    kf = np.array([0, 0, 1])[:, None]
    q = kf - ki_rotated
    print(q)
    q_norm_all[i] = np.linalg.norm(q)

0
[[ 1.          0.          0.        ]
 [ 0.          0.98432049 -0.17638924]
 [ 0.          0.17638924  0.98432049]]
[[ 2.02572732e-17]
 [-3.47247097e-01]
 [ 6.22263312e-02]]
1
[[ 1.          0.          0.        ]
 [ 0.          0.93570687 -0.35277849]
 [ 0.          0.35277849  0.93570687]]
[[ 0.33009725]
 [-0.35277849]
 [ 0.12445266]]
2
[[ 1.          0.          0.        ]
 [ 0.          0.47135004 -0.88194622]
 [ 0.          0.88194622  0.47135004]]
[[ 0.68585857]
 [-0.23464532]
 [ 0.31113166]]
3
[[ 1.          0.          0.        ]
 [ 0.          0.89752031 -0.44097311]
 [ 0.          0.44097311  0.89752031]]
[[ 0.25971588]
 [-0.67676293]
 [ 0.31113166]]


In [38]:
scalings_all = 1e-3 / q_norm_all**4
scalings_all

array([0.06456418, 0.01614104, 0.00258257, 0.00258257])

In [43]:
scalings_all / 8

array([0.00807052, 0.00201763, 0.00032282, 0.00032282])

In [44]:
scalings_all / 4

array([0.01614104, 0.00403526, 0.00064564, 0.00064564])